# Estudiantes - PySpark

 ## 1. Carga e inspección del dataset

Objetivo
El objetivo principal de este trabajo es analizar el impacto de diversos factores personales, académicos y conductuales en los niveles de depresión de los estudiantes, utilizando PySpark como herramienta de procesamiento y análisis de datos.
Los objetivos específicos incluyen:
Identificar correlaciones entre el rendimiento académico y los niveles de depresión.


Evaluar cómo hábitos como el sueño, el ejercicio o las relaciones sociales influyen en el estado emocional de los estudiantes.


Promover el uso responsable de datos sensibles, priorizando la ética en el análisis.


Columnas	
Está formado por varias columnas:
- id: ID Identificación de cada estudiante
- Gender: Género de cada estudiantes (Hombre,Mujer)
- Age: Edad de cada estudiantes
- City: Ciudad de cada estudiante
- Profession: Profesión de cada estudiante(solo estudiante)
- Academic Pressure: Presión académica que es la medida que se tiene en cuenta si
un estudiante sufre mucha presión en sus estudios.
- Work Pressure: Presión laboral.
- CGPA: Promedio General de Calificaciones (CGPA, por sus siglas en inglés)
- Study Satisfaction: Satisfacción con el estudio
- Job Satisfaction: Satisfacción laboral
- Sleep Duration: Duración del sueño medio de cada estudiante
- Dietary Habits: Hábitos alimenticios de cada estudiante
- Degree: Título (o Grado académico)
- Have you ever had suicidal thoughts?: ¿Alguna vez has tenido pensamientos
suicidas?
- Work/Study Hours: Horas de trabajo/estudio de media de cada estudiante
- Financial Stress: Estrés financiero de cada estudiante
- Family History of Mental Illness: Antecedentes familiares de enfermedades
mentales
- Depression: Depresión si la sufre actualmente o no.


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("estudiantes").getOrCreate()

# Cargar el CSV
df = spark.read.option("header", "true").option("inferSchema", "true").csv("s3://jmgarcia-spark/datasets/StudentDepressionDataset_sucio.csv")

# Mostrar esquema
df.printSchema()

# Mostrar primeros registros
df.show(5, truncate=False)

# Contar registros
print("Número total de registros:", df.count())


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Profession: string (nullable = true)
 |-- Academic Pressure: double (nullable = true)
 |-- Work Pressure: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Study Satisfaction: double (nullable = true)
 |-- Job Satisfaction: double (nullable = true)
 |-- Sleep Duration: string (nullable = true)
 |-- Dietary Habits: string (nullable = true)
 |-- Degree: string (nullable = true)
 |-- Have you ever had suicidal thoughts ?: string (nullable = true)
 |-- Work/Study Hours: double (nullable = true)
 |-- Financial Stress: double (nullable = true)
 |-- Family History of Mental Illness: string (nullable = true)
 |-- Depression: integer (nullable = true)

+---+------+----+-------------+----------+-----------------+-------------+----+------------------+----------------+-----------------+--------------+-------+----------------------

## 2. Análisis de calidad de datos

In [7]:
from pyspark.sql.functions import col, when, count, sum as sum_
# Lista para guardar los resultados
nulos = []

# Recorre cada columna y cuenta los nulos
for colums_name in df.columns:
    nulo_count = df.select(count(when(col(colums_name).isNull(), colums_name)).alias("nulos")).collect()[0]["nulos"]
    nulos.append((colums_name, nulo_count))

# Creamos un nuevo DataFrame con los resultados de los nulos por cada columna
df_denulos = spark.createDataFrame(nulos, ["columna", "nulos"])
df_denulos.show(truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------+-----+
|columna                              |nulos|
+-------------------------------------+-----+
|id                                   |0    |
|Gender                               |0    |
|Age                                  |9    |
|City                                 |0    |
|Profession                           |0    |
|Academic Pressure                    |0    |
|Work Pressure                        |0    |
|CGPA                                 |13   |
|Study Satisfaction                   |0    |
|Job Satisfaction                     |0    |
|Sleep Duration                       |0    |
|Dietary Habits                       |0    |
|Degree                               |0    |
|Have you ever had suicidal thoughts ?|0    |
|Work/Study Hours                     |0    |
|Financial Stress                     |3    |
|Family History of Mental Illness     |0    |
|Depression                           |0    |
+---------------------------------

## 3.Limpieza de datos

In [8]:
# Rellenar valores nulos
from pyspark.sql.functions import percentile_approx
mediana_age = df.select(percentile_approx("Age", 0.5)).first()[0]
mediana_cgpa = df.select(percentile_approx("CGPA", 0.5)).first()[0]
# Rellenar valores
df_sin_nulos = df.fillna({
    "Age": mediana_age,
    "CGPA": mediana_cgpa,
})
print("Registros limpios: ",df_sin_nulos.count())
print("Registros eliminados: ",df.count() - df_sin_nulos.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Registros limpios:  27901
Registros eliminados:  0

In [9]:
#Eliminar los nulos restantes que son 3 de la columna Financial Stress
df_sin_nulos = df_sin_nulos.na.drop(subset=["Financial Stress"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
print("Registros limpios: ",df_sin_nulos.count())
print("Registros eliminados: ",df.count() - df_sin_nulos.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Registros limpios:  27898
Registros eliminados:  3

In [11]:
# De algunas columnas categóricas eliminar espacios en blanco y pasar todo a minúsculas
from pyspark.sql.functions import year, trim, lower, when
df_sin_nulos = df_sin_nulos.withColumn("City", lower(trim(col("City")))).withColumn("Sleep Duration", lower(trim(col("Sleep Duration")))).withColumn("Degree", lower(trim(col("Degree"))))
df_sin_nulos.limit(5).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------+----+-------------+----------+-----------------+-------------+----+------------------+----------------+-----------------+--------------+-------+-------------------------------------+----------------+----------------+--------------------------------+----------+
| id|Gender| Age|         City|Profession|Academic Pressure|Work Pressure|CGPA|Study Satisfaction|Job Satisfaction|   Sleep Duration|Dietary Habits| Degree|Have you ever had suicidal thoughts ?|Work/Study Hours|Financial Stress|Family History of Mental Illness|Depression|
+---+------+----+-------------+----------+-----------------+-------------+----+------------------+----------------+-----------------+--------------+-------+-------------------------------------+----------------+----------------+--------------------------------+----------+
|  2|  Male|33.0|visakhapatnam|   Student|              5.0|          0.0|7.77|               2.0|             0.0|        5-6 hours|       Healthy|b.pharm|                         

## 4. Transformaciones

In [12]:
#Crea una nueva columnas con ceros y unos si el estudiante tiene un alto riesgo de tener algún problema mental que afecte al rendimiento académico
df_estudiantes = df_sin_nulos.withColumn("HighRisk",
    when((col("Academic Pressure") >= 4) &
         (col("Sleep Duration") == "less than 5 hours") &
         (col("Financial Stress") >= 4.0) &
         (col("Family History of Mental Illness") == "Yes"), 1)
    .otherwise(0)
)
df_estudiantes.select("id","Gender","HighRisk").limit(5).show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------+--------+
| id|Gender|HighRisk|
+---+------+--------+
|  2|  Male|       0|
|  8|Female|       0|
| 26|  Male|       0|
| 30|Female|       0|
| 32|Female|       0|
+---+------+--------+

In [13]:
#Crea una nueva columnas para indicar si el estudiante tiene buenos hábitos o no
df_estudiantes = df_estudiantes.withColumn("GoodHabits",
    when(
        (col("Sleep Duration") == "7-8 hours") &
        (col("Dietary Habits").isin("Healthy", "Moderate")) &
        (col("Work/Study Hours") >= 3) & (col("Work/Study Hours") <= 8),
        1
    ).otherwise(0)
)
df_estudiantes.select("id","Gender","GoodHabits").limit(5).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------+----------+
| id|Gender|GoodHabits|
+---+------+----------+
|  2|  Male|         0|
|  8|Female|         0|
| 26|  Male|         0|
| 30|Female|         1|
| 32|Female|         0|
+---+------+----------+

In [14]:
# Ayuda después para que no se tenga que filtar si es mayor que un número o no. Lo que quiero que ya indique si tiene estres financiero o no.
df_estudiantes = df_estudiantes.withColumn("HasFinancialStress", when(col("Financial Stress") >= 3.0, 1).otherwise(0))
df_estudiantes = df_estudiantes.withColumn("HasAcademicPressure", when(col("Academic Pressure") >= 3.0, 1).otherwise(0))
df_estudiantes = df_estudiantes.withColumn("HasSatisfaction", when(col("Study Satisfaction") >= 3.0, 1).otherwise(0))

# Cambiar los datos de si y no a binario (0 y 1)
df_estudiantes = df_estudiantes.withColumn("HasSuicidalThoughts", when(col("Have you ever had suicidal thoughts ?") == "Yes", 1).otherwise(0))
df_estudiantes = df_estudiantes.withColumn("HasFamilyHistory", when(col("Family History of Mental Illness") == "Yes", 1).otherwise(0))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_estudiantes.select("HasFinancialStress","HasAcademicPressure","HasSatisfaction","HasSuicidalThoughts","HasFamilyHistory").limit(5).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-------------------+---------------+-------------------+----------------+
|HasFinancialStress|HasAcademicPressure|HasSatisfaction|HasSuicidalThoughts|HasFamilyHistory|
+------------------+-------------------+---------------+-------------------+----------------+
|                 0|                  1|              0|                  1|               0|
|                 0|                  0|              1|                  0|               1|
|                 0|                  1|              1|                  0|               1|
|                 1|                  1|              0|                  1|               1|
|                 0|                  1|              1|                  1|               0|
+------------------+-------------------+---------------+-------------------+----------------+

In [16]:
#Pasar a booleano tanto la edad como el género para ayudar al entrenamiento
#Crea una nueva columna para indicar si el estudiantes es joven o no
df_estudiantes = df_estudiantes.withColumn("IsYoung", when((col("Age") >= 18) & (col("Age") <= 25), 1).otherwise(0))
#Crea una nueva columna para indicar si el estudiantes es mujer u hombre
df_estudiantes = df_estudiantes.withColumn("IsMale", when((col("Gender") == "Male"), 1).otherwise(0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df_estudiantes.select("IsYoung","IsMale").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+
|IsYoung|IsMale|
+-------+------+
|      0|     1|
|      1|     0|
|      0|     1|
|      0|     0|
|      1|     0|
+-------+------+
only showing top 5 rows

In [18]:
#Crea una nueva columna para indicar si el alumno tiene malos resultados o no
df_estudiantes = df_estudiantes.withColumn("BadCGPA", when(col("CGPA") < 6.0, 1).otherwise(0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_estudiantes.select("id","CGPA","BadCGPA").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----+-------+
| id|CGPA|BadCGPA|
+---+----+-------+
|  2|7.77|      0|
|  8| 5.9|      1|
| 26|7.03|      0|
| 30|7.77|      0|
| 32|8.13|      0|
+---+----+-------+
only showing top 5 rows

## 5. Validación

In [20]:
#Crea un nuevo dataframe para comprobar que el dataframe original no tiene nulos
# Lista para guardar los resultados
nulos = []

# Recorre cada columna y cuenta los nulos
for colums_name in df_estudiantes.columns:
    nulo_count = df_estudiantes.select(count(when(col(colums_name).isNull(), colums_name)).alias("nulos")).collect()[0]["nulos"]
    nulos.append((colums_name, nulo_count))

# Creamos un nuevo DataFrame con los resultados
df_denulos = spark.createDataFrame(nulos, ["columna", "nulos"])
df_denulos.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------+-----+
|columna                              |nulos|
+-------------------------------------+-----+
|id                                   |0    |
|Gender                               |0    |
|Age                                  |0    |
|City                                 |0    |
|Profession                           |0    |
|Academic Pressure                    |0    |
|Work Pressure                        |0    |
|CGPA                                 |0    |
|Study Satisfaction                   |0    |
|Job Satisfaction                     |0    |
|Sleep Duration                       |0    |
|Dietary Habits                       |0    |
|Degree                               |0    |
|Have you ever had suicidal thoughts ?|0    |
|Work/Study Hours                     |0    |
|Financial Stress                     |0    |
|Family History of Mental Illness     |0    |
|Depression                           |0    |
|HighRisk                         

In [21]:
# Compruebo los valores únicos de cada columna en el dataframe final
for columns_name in df_estudiantes.columns:
    print(f"Valores únicos de las columnas {columns_name}")
    df_estudiantes.select(columns_name).distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Valores ?nicos de las columnas id
+-----+
|   id|
+-----+
| 1591|
| 4519|
|11141|
|11317|
|12046|
|16339|
|16503|
|24663|
|25591|
|26583|
|26623|
|32445|
|36224|
|43302|
|43527|
|45307|
|57984|
|72758|
|78120|
|81900|
+-----+
only showing top 20 rows

Valores ?nicos de las columnas Gender
+------+
|Gender|
+------+
|  Male|
|Female|
+------+

Valores ?nicos de las columnas Age
+----+
| Age|
+----+
|29.0|
|46.0|
|28.0|
|26.0|
|21.0|
|48.0|
|27.0|
|20.0|
|54.0|
|43.0|
|42.0|
|44.0|
|35.0|
|37.0|
|25.0|
|41.0|
|23.0|
|56.0|
|51.0|
|19.0|
+----+
only showing top 20 rows

Valores ?nicos de las columnas City
+-------------+
|         City|
+-------------+
|       bhavna|
|       bhopal|
|    ahmedabad|
|       m.tech|
|        rashi|
|       saanvi|
|        patna|
|     srinagar|
|       nalyan|
|       kalyan|
|       jaipur|
|     ludhiana|
|     vadodara|
|        thane|
|       nashik|
|        delhi|
|      lucknow|
|visakhapatnam|
|  vasai-virar|
|        surat|
+-------------+
only s

## 6. Persistencia

In [22]:
# Una vez realizado el proceso de limpieza y tranformaciones que he considerado oportunas
# Guardo el dataset en s3 con formato parquet para ahorrar.
df_estudiantes.write.mode("overwrite").parquet("s3://jmgarcia-spark/datasets/outputs/estudiantesdepresion_limpio.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 7.  Investigación MLlib

Un dataset limpio y bien transformado es el punto de partida ideal para entrenar un modelo de IA. 

### Descripción de la biblioteca

MLlib es la biblioteca de aprendizaje automático (ML) de Spark. Su objetivo es que el aprendizaje automático práctico sea escalable y sencillo. A grandes rasgos, proporciona herramientas como:

- Algoritmos de aprendizaje automático: algoritmos de aprendizaje comunes como clasificación, regresión, agrupamiento y filtrado colaborativo.
- Caracterización: extracción de características, transformación, reducción de dimensionalidad y selección
- Pipelines: herramientas para construir, evaluar y ajustar pipelines de ML
- Persistencia: guardar y cargar algoritmos, modelos y pipelines
- Utilidades: álgebra lineal, estadística, manejo de datos, etc.

In [23]:
# Dataset para entrenar el modelo
df_estudiantes.show(truncate=False,n=10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------+----+-------------+----------+-----------------+-------------+----+------------------+----------------+-----------------+--------------+--------+-------------------------------------+----------------+----------------+--------------------------------+----------+--------+----------+------------------+-------------------+---------------+-------------------+----------------+-------+------+-------+
|id |Gender|Age |City         |Profession|Academic Pressure|Work Pressure|CGPA|Study Satisfaction|Job Satisfaction|Sleep Duration   |Dietary Habits|Degree  |Have you ever had suicidal thoughts ?|Work/Study Hours|Financial Stress|Family History of Mental Illness|Depression|HighRisk|GoodHabits|HasFinancialStress|HasAcademicPressure|HasSatisfaction|HasSuicidalThoughts|HasFamilyHistory|IsYoung|IsMale|BadCGPA|
+---+------+----+-------------+----------+-----------------+-------------+----+------------------+----------------+-----------------+--------------+--------+-------------------------

### Objetivo del entrenamiento

El objetivo de este entrenamiento es predecir si un estudiante sufre depresion o no.

In [28]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# Selección de características para el modelo
assembler = VectorAssembler(
    inputCols=["HasFinancialStress", "HasAcademicPressure", "HasSatisfaction",
               "HasSuicidalThoughts", "HasFamilyHistory", "IsYoung", "IsMale", "BadCGPA","GoodHabits"],
    outputCol="data")
df = assembler.transform(df_estudiantes)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# Dividir en conjunto de entrenamiento y prueba
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# Definir y entrenar el modelo
lr = LogisticRegression(featuresCol='data', labelCol='Depression')
model = lr.fit(train_data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# Realizar predicciones
predictions = model.transform(test_data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
# Evaluar el modelo
evaluator = MulticlassClassificationEvaluator(
    labelCol="Depression", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Precisión del modelo: {accuracy:.4f}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Precisi?n del modelo: 0.8164

In [36]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…